In [ ]:
from D3QN import DuelingDeepQNetwork
import torch
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [ ]:
ACTIONS= ["IDLE", "L1", "L2", "L3"]

In [ ]:
cache_agent = DuelingDeepQNetwork(alpha=1e-3,
                              state_dim=15,
                              action_dim=len(ACTIONS),
                              fc1_dim=64,
                              fc2_dim=32)


class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = []
        self.labels = []
        self.gen()

    def gen(self):
        pass

    def cold_start(self):
        obs = {
            "ES_load":random.uniform(0, 0.1),
            "free_storage_size_ratio_L1":random.uniform(0, 0.1),
            "free_storage_size_ratio_L2":random.uniform(0.9, 1),
            "free_storage_size_ratio_L3":random.uniform(0.9, 1),
            "can_L1_fit":1,
            "can_L2_fit":1,
            "can_L3_fit":1,
            "estimated_speed":random.uniform(5, 100),
            "estimated_fetch_time":random.uniform(1, 60),
            "is_popular":0,
            "charm":round(abs(np.random.standard_normal(1)[0]), 2),
            "service_request_frequency":random.randint(0, 3),
            "nearby_servers_count":random.uniform(0, 1),
            "cached_in_nearby_servers":0,
            "ES_request_frequency":random.uniform(0, 1)
        }
        return obs, "L1"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]


def train(network, epochs):
    loss_func = torch.nn.CrossEntropyLoss()
    dataset = MyDataset()
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    optimizer = network.optimizer
    for i in range(epochs):
        for data, labels in dataloader:
            optimizer.zero_grad()
            output = network(data)
            loss = loss_func(output, labels)
            loss.backward()
            optimizer.step()
            print(loss)


train(cache_agent, 10)
cache_agent.save_checkpoint('test.pth')
